In [1]:
import json
import random
from pathlib import Path

# !pip install camelot
# import camelot

# !pip install lxml
import pandas as pd

%load_ext autoreload
%autoreload 2
Path.ls = lambda x: list(x.iterdir())

In [2]:
covid19india_mirror_url = "https://api.steinhq.com/v1/storages/5e737e60b88d3d04ae0815b8/Sheet1"
ka_url = "https://api.steinhq.com/v1/storages/5e7c53b3b88d3d04ae0815f0/KAPatientTravelHistory"

In [3]:
df = pd.read_json(ka_url)
df.columns = [c.strip() for c in df.columns] # remove newline, space etc noise

## Location Verification

In [6]:
def maps_link_parser(link: str):
    if link is None:
        return [None, None, None]
    if link.strip().upper() == "UNKNOWN":
        return [None, None, None]
    split_link = link.split("/")
#     print(split_link)
    name_place = split_link[5].replace("+", " ")
    try:
        lat, long, _ = split_link[6].replace("@", "").split(",")
    except:
        return ["ParsingError", "ParsingError", "ParsingError"]
    return [name_place, lat, long]

place_name, place_lat, place_long = maps_link_parser(df["GoogleMapsLink"][2])

In [7]:
new_df = df["GoogleMapsLink"].apply(maps_link_parser)

In [8]:
new_df = pd.DataFrame(new_df.tolist(), columns=["PlaceName", "Latitude", "Longitude"])

In [9]:
new_df

,PlaceName,Latitude,Longitude
0,Austin-Bergstrom International Airport,30.1974711,-97.6663529
1,John F. Kennedy International Airport,40.6413151,-73.7803278
2,Dubai International Airport,25.2531793,55.3634841
3,Kempegowda International Airport Bengaluru,13.19864,77.7044041
4,"Whitefield, Bengaluru, Karnataka",12.9646087,77.7190231
...,...,...,...
134,"Thanisandra, Bengaluru, Karnataka",13.0582033,77.6159807
135,"Thanisandra, Bengaluru, Karnataka",13.0582033,77.6159807
136,"Rahmath Nagar, RT Nagar, Bengaluru, Karnataka ...",13.0186044,77.5920903
137,ESI Hospital,12.9711854,77.6347212


In [10]:
df["PlaceName"] = new_df["PlaceName"]
df["Lat"] = new_df.Latitude
df["Long"] = new_df.Longitude

## DateTime Parsing

In [11]:
from dateutil.parser import parse
from datetime import datetime
def insert(source_str, insert_str, pos):
    return source_str[:pos]+insert_str+source_str[pos:]

def parse_date_time(date_time_str:str)->datetime:
    if date_time_str is None:
        return None
#     print(date_time_str)
    split_str = date_time_str.split()
    if len(split_str) == 1:
        return parse(date_time_str)
    date_str = split_str[0]
    time_str = insert(split_str[1], ":", 2)
    return parse(date_str + " " + time_str)
    
# sample_date_time = df["From"][11]
# sample_date_time, parse_date_time(sample_date_time)

In [12]:
df["To"] = df["To"].apply(parse_date_time)
df["From"] = df["From"].apply(parse_date_time)

In [16]:
fname = ka_url.split("/")[-1]
df.to_csv(f"{fname}.csv", index=False)